<div class="alert alert-info" style="color:black" > <h1> Activity 4: Creating a test harness for comparing ML algorithms on a dataset</h1>
<p> Now you have done some manual experimenting with different hyper-parameter values for algorithms, it's time to think about automating that process.</p>
<p>Complete the cell below to create a method that: </p>
<ul>
    <li> Takes a train and test data  arrays as  parameters <br>
        HINT: develop your code using train_x, test_x,train_y,test_y for the iris data from above</li>
    <li> Runs your SimpleKNNClassifier with K={1,3,5,7,9} and stores the test accuracy for each <br>
    HINT: you could use:
        <ul>
            <li>a for loop to run the algorithm with different settings k  for  the number of neighbours(K),</li>
            <li> an <a href=https://www.geeksforgeeks.org/formatted-string-literals-f-strings-python/>f-string</a> e.g. <code>experiment_name= f'KNN_K={k}'</code> to create a meaningful name for each run </li>
            <li>a   dictionary to store your results, where each experiment has the string <em>experiment_name</em> as the key and the accuracy as the value </li>
            </ul>for this?</li>
    <li> Runs a DecisionTreeClassifier with all the different combinations of hyper-parameters from activity 3<br>
       HINT: You could do this in the same way as I've suggested above but with nested for-loops (one for each hyper-parameter) and a more complex python f-string to create the name (key), then store the results in the same dictionary.  </li>
    <li> Reports the results and which algorithm-hyperparameter combination has the highest test accuracy</li>
</ul>
</div>

<div class="alert alert-warning" style="color:black"">
    <h3> Reminder: Storing data in python dictionaries and iterating through their contents</h3>
    <p> Python dictionaries are a way of storing data that can be accessed via a key<br>
for example: <code> my_dict= {'name':'jim','familyname':"Smith", 'job':'professor'}</code><br>
<b>Keys are usually strings</b>, but the values associated with a key can be any type, including numbers.</p>

<p> The following snippets of code might be useful to you - <b>after</b> you have edited them.</p>
<p> Make a new code cell in the notebook, copy the snippets in and run it, then edit it as you need.</p>
<p><pre style='background:lightbrown;colour:black'>    
labels = ['a','b','a','c','a','d','b']
indexes = [1,4,6]
mydict={}
<span style="color:green">for</span> idx <span style="color:green">in</span> indexes:
    <span style="color:green">if</span> labels[idx] <span style="color:green">in</span> mydict.keys():
        mydict[labels[idx]] += 1
    <span style="color:green">else</span>: #create a new dictionary entry if needed
        mydict[labels[idx]] = 1
<span style="color:green">print</span>(f'mydict is {mydict}')

leastvotes=99
<span style="color:green">for</span> key,val <span style="color:green">in</span> mydict.items():
    <span style="color:green">if</span> val < leastvotes:
        unpopular= key
        leastvotes=val
<span style="color:green">print</span>(f'{unpopular}, {leastvotes}')
    </pre></p>
    </div>

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import numpy as np

# Load iris dataset
iris = load_iris()
irisX = iris.data
irisy = iris.target

# Split into train and test sets
train_x, test_x, train_y, test_y = train_test_split(irisX, irisy, 
                                                   test_size=0.33, 
                                                   stratify=irisy, 
                                                   random_state=42)
class SimpleKNNClassifier:
    def __init__(self, k=3):
        self.k = k
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
    def predict(self, X):
        predictions = []
        for row in X:
            label = self._predict(row)
            predictions.append(label)
        return np.array(predictions)
    
    def _predict(self, x):
        # Compute distances
        distances = [np.linalg.norm(x - x_train) for x_train in self.X_train]
        # Get k nearest neighbors
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        # Majority vote
        most_common = np.bincount(k_nearest_labels).argmax()
        return most_common
    
def first_ml_test_harness(train_x:np.ndarray,train_y:np.ndarray,
                          test_x:np.ndarray=None,test_y:np.ndarray=None):
    """ code to compare supervised machine learning algorithms on a dataset"""
    results = {}
    
    if test_x is None or test_y is None:
        test_x, test_y = train_x, train_y
    
    # Test SimpleKNNClassifier with different K values
    for k in [1, 3, 5, 7, 9]:
        knn = SimpleKNNClassifier(k=k)
        knn.fit(train_x, train_y)
        pred_y = knn.predict(test_x)
        accuracy = np.mean(pred_y == test_y)
        experiment_name = f'KNN_K={k}'
        results[experiment_name] = accuracy
    
    # Test DecisionTreeClassifier with different hyper-parameters
    for max_depth in [None, 2, 4, 6]:
        for min_samples_split in [2, 4, 6]:
            for criterion in ['gini', 'entropy']:
                dt = DecisionTreeClassifier(max_depth=max_depth, 
                                          min_samples_split=min_samples_split,
                                          criterion=criterion)
                dt.fit(train_x, train_y)
                pred_y = dt.predict(test_x)
                accuracy = np.mean(pred_y == test_y)
                experiment_name = f'DT_depth={max_depth}_split={min_samples_split}_crit={criterion}'
                results[experiment_name] = accuracy
    
    # Report all results
    for experiment, accuracy in results.items():
        print(f"{experiment}: {accuracy:.4f}")
    
    best_experiment = max(results, key=results.get)
    best_accuracy = results[best_experiment]
    print(f"\nBest configuration: {best_experiment} with accuracy {best_accuracy:.4f}")
    
    return results

first_ml_test_harness(train_x, train_y, test_x, test_y)

KNN_K=1: 0.9400
KNN_K=3: 0.9600
KNN_K=5: 0.9800
KNN_K=7: 0.9800
KNN_K=9: 0.9600
DT_depth=None_split=2_crit=gini: 0.9800
DT_depth=None_split=2_crit=entropy: 0.9200
DT_depth=None_split=4_crit=gini: 0.9400
DT_depth=None_split=4_crit=entropy: 0.9000
DT_depth=None_split=6_crit=gini: 0.9400
DT_depth=None_split=6_crit=entropy: 0.9000
DT_depth=2_split=2_crit=gini: 0.9000
DT_depth=2_split=2_crit=entropy: 0.9000
DT_depth=2_split=4_crit=gini: 0.9000
DT_depth=2_split=4_crit=entropy: 0.9000
DT_depth=2_split=6_crit=gini: 0.9000
DT_depth=2_split=6_crit=entropy: 0.9000
DT_depth=4_split=2_crit=gini: 0.9400
DT_depth=4_split=2_crit=entropy: 0.9200
DT_depth=4_split=4_crit=gini: 0.9400
DT_depth=4_split=4_crit=entropy: 0.9000
DT_depth=4_split=6_crit=gini: 0.9400
DT_depth=4_split=6_crit=entropy: 0.9200
DT_depth=6_split=2_crit=gini: 0.9400
DT_depth=6_split=2_crit=entropy: 0.9400
DT_depth=6_split=4_crit=gini: 0.9400
DT_depth=6_split=4_crit=entropy: 0.9000
DT_depth=6_split=6_crit=gini: 0.9400
DT_depth=6_split=6

{'KNN_K=1': np.float64(0.94),
 'KNN_K=3': np.float64(0.96),
 'KNN_K=5': np.float64(0.98),
 'KNN_K=7': np.float64(0.98),
 'KNN_K=9': np.float64(0.96),
 'DT_depth=None_split=2_crit=gini': np.float64(0.98),
 'DT_depth=None_split=2_crit=entropy': np.float64(0.92),
 'DT_depth=None_split=4_crit=gini': np.float64(0.94),
 'DT_depth=None_split=4_crit=entropy': np.float64(0.9),
 'DT_depth=None_split=6_crit=gini': np.float64(0.94),
 'DT_depth=None_split=6_crit=entropy': np.float64(0.9),
 'DT_depth=2_split=2_crit=gini': np.float64(0.9),
 'DT_depth=2_split=2_crit=entropy': np.float64(0.9),
 'DT_depth=2_split=4_crit=gini': np.float64(0.9),
 'DT_depth=2_split=4_crit=entropy': np.float64(0.9),
 'DT_depth=2_split=6_crit=gini': np.float64(0.9),
 'DT_depth=2_split=6_crit=entropy': np.float64(0.9),
 'DT_depth=4_split=2_crit=gini': np.float64(0.94),
 'DT_depth=4_split=2_crit=entropy': np.float64(0.92),
 'DT_depth=4_split=4_crit=gini': np.float64(0.94),
 'DT_depth=4_split=4_crit=entropy': np.float64(0.9),
 

In [9]:
import numpy as np

#some names
people= ['jim', 'nathan']
RUNS= 5
#some randomly created scores
scores= [[46,23,69,12,78], [79, 13,48,63,39]]

#some example of making keys and putting things into  dict
mydict={}
#populate mydict with items created using the data above
for iteration in range(RUNS):
    for person_idx in range(2):
        name= f'{people[person_idx]}_{iteration}'
        mydict[name]= scores[person_idx][iteration]

#some examples of accessing a dictionary's contents
for key,val in mydict.items():
    print( f'{key} : {val}')

# print all scores
vals= mydict.values()
print(f' mydict.values() returns {vals} of type {type(vals)}')

# Have to cast vals into a list, then make a numpy array from that, to do maths
num_vals= np.array(list(vals))
print(f' biggest value is {np.max(num_vals)}') 

#print all keys
print(mydict.keys())


jim_0 : 46
nathan_0 : 79
jim_1 : 23
nathan_1 : 13
jim_2 : 69
nathan_2 : 48
jim_3 : 12
nathan_3 : 63
jim_4 : 78
nathan_4 : 39
 mydict.values() returns dict_values([46, 79, 23, 13, 69, 48, 12, 63, 78, 39]) of type <class 'dict_values'>
 biggest value is 79
dict_keys(['jim_0', 'nathan_0', 'jim_1', 'nathan_1', 'jim_2', 'nathan_2', 'jim_3', 'nathan_3', 'jim_4', 'nathan_4'])


In [ ]:
def first_ml_test_harness(train_x:np.ndarray,train_y:np.ndarray,
                          test_x:np.ndarray=None,test_y:np.ndarray=None):
    """ code to compare supervised machine learning algorithms on a dataset"""
    # insert your code below here
    
    results = {}

    if test_x is None or test_y is None:
        test_x, test_y = train_x, train_y
    
    for k in [1, 3, 5, 7, 9]:
        knn = SimpleKNNClassifier(k=k)
        knn.fit(train_x, train_y)
        pred_y = knn.predict(test_x)
        accuracy = np.mean(pred_y == test_y)
        experiment_name = f'KNN_K={k}'
        results[experiment_name] = accuracy
    
    for max_depth in [None, 2, 4, 6]:
        for min_samples_split in [2, 4, 6]:
            for criterion in ['gini', 'entropy']:
                dt = DecisionTreeClassifier(max_depth=max_depth, 
                                          min_samples_split=min_samples_split,
                                          criterion=criterion)
                dt.fit(train_x, train_y)
                pred_y = dt.predict(test_x)
                accuracy = np.mean(pred_y == test_y)
                experiment_name = f'DT_depth={max_depth}_split={min_samples_split}_crit={criterion}'
                results[experiment_name] = accuracy
    
    # Report all results
    for experiment, accuracy in results.items():
        print(f"{experiment}: {accuracy:.4f}")
    
    # Find and report the best performing configuration
    best_experiment = max(results, key=results.get)
    best_accuracy = results[best_experiment]
    print(f"\nBest configuration: {best_experiment} with accuracy {best_accuracy:.4f}")
    
    return results
    #insert your code above here

In [11]:
#now run your code for the iris data
first_ml_test_harness(train_x, train_y)

KNN_K=1: 1.0000
KNN_K=3: 0.9600
KNN_K=5: 0.9600
KNN_K=7: 0.9700
KNN_K=9: 0.9700
DT_depth=None_split=2_crit=gini: 1.0000
DT_depth=None_split=2_crit=entropy: 1.0000
DT_depth=None_split=4_crit=gini: 0.9800
DT_depth=None_split=4_crit=entropy: 0.9900
DT_depth=None_split=6_crit=gini: 0.9800
DT_depth=None_split=6_crit=entropy: 0.9900
DT_depth=2_split=2_crit=gini: 0.9700
DT_depth=2_split=2_crit=entropy: 0.9700
DT_depth=2_split=4_crit=gini: 0.9700
DT_depth=2_split=4_crit=entropy: 0.9700
DT_depth=2_split=6_crit=gini: 0.9700
DT_depth=2_split=6_crit=entropy: 0.9700
DT_depth=4_split=2_crit=gini: 0.9900
DT_depth=4_split=2_crit=entropy: 0.9900
DT_depth=4_split=4_crit=gini: 0.9800
DT_depth=4_split=4_crit=entropy: 0.9900
DT_depth=4_split=6_crit=gini: 0.9800
DT_depth=4_split=6_crit=entropy: 0.9900
DT_depth=6_split=2_crit=gini: 1.0000
DT_depth=6_split=2_crit=entropy: 1.0000
DT_depth=6_split=4_crit=gini: 0.9800
DT_depth=6_split=4_crit=entropy: 0.9900
DT_depth=6_split=6_crit=gini: 0.9800
DT_depth=6_split=6

{'KNN_K=1': np.float64(1.0),
 'KNN_K=3': np.float64(0.96),
 'KNN_K=5': np.float64(0.96),
 'KNN_K=7': np.float64(0.97),
 'KNN_K=9': np.float64(0.97),
 'DT_depth=None_split=2_crit=gini': np.float64(1.0),
 'DT_depth=None_split=2_crit=entropy': np.float64(1.0),
 'DT_depth=None_split=4_crit=gini': np.float64(0.98),
 'DT_depth=None_split=4_crit=entropy': np.float64(0.99),
 'DT_depth=None_split=6_crit=gini': np.float64(0.98),
 'DT_depth=None_split=6_crit=entropy': np.float64(0.99),
 'DT_depth=2_split=2_crit=gini': np.float64(0.97),
 'DT_depth=2_split=2_crit=entropy': np.float64(0.97),
 'DT_depth=2_split=4_crit=gini': np.float64(0.97),
 'DT_depth=2_split=4_crit=entropy': np.float64(0.97),
 'DT_depth=2_split=6_crit=gini': np.float64(0.97),
 'DT_depth=2_split=6_crit=entropy': np.float64(0.97),
 'DT_depth=4_split=2_crit=gini': np.float64(0.99),
 'DT_depth=4_split=2_crit=entropy': np.float64(0.99),
 'DT_depth=4_split=4_crit=gini': np.float64(0.98),
 'DT_depth=4_split=4_crit=entropy': np.float64(0.